In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.utils import resample
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, FunctionTransformer, Imputer
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier 

%matplotlib inline


In [2]:
df = pd.read_csv('../data/formula1.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train = df[(df['year'] < 2016)]
test = df[(df['year'] >= 2016)]

In [4]:
train.shape, test.shape

((9946, 158), (862, 158))

In [5]:
# Take a look at the numeric variables:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,9946.0,5.424770e+03,3.097687e+03,0.0,2732.25,5464.5,8.196750e+03,1.076500e+04
resultId,9946.0,8.393005e+03,8.103317e+03,1.0,2487.25,4973.5,2.043075e+04,2.354100e+04
raceId,9946.0,3.337168e+02,3.040892e+02,1.0,121.00,235.0,3.410000e+02,9.450000e+02
driverId,9946.0,1.282593e+02,2.402700e+02,1.0,17.00,44.0,8.800000e+01,8.340000e+02
Team_Code,9946.0,2.829791e+01,4.964069e+01,1.0,5.00,15.0,2.200000e+01,2.090000e+02
grid,9946.0,1.145445e+01,6.766506e+00,0.0,6.00,11.0,1.700000e+01,2.800000e+01
position,9946.0,5.507239e+00,5.670649e+00,0.0,0.00,4.0,1.000000e+01,2.400000e+01
points,9946.0,2.171677e+00,4.466974e+00,0.0,0.00,0.0,2.000000e+00,5.000000e+01
laps,9946.0,4.763091e+01,2.275179e+01,0.0,35.00,55.0,6.500000e+01,8.300000e+01
milliseconds,9946.0,1.986705e+06,2.800253e+06,0.0,0.00,0.0,5.376580e+06,1.474314e+07


MODELING

In [6]:
corrs = train.corr()[['rank']]
corrs['abs_corr']=abs(corrs['rank'])
corrs.sort_values('abs_corr', ascending=False).drop('abs_corr', axis=1).head(30)

,rank
rank,1.000000
year,0.670802
fastestLap,0.614947
dob,0.590114
position_y,0.572754
qualifyId,0.485861
constructorId_y,0.436800
resultId,0.417373
position,0.407357
Team_Code,0.360443


REGRESSION


In [21]:
y = train[['rank']]
X = train.drop('rank', axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, random_state=418, stratify = y)

In [22]:
features = list(corrs[corrs['abs_corr'] > .09].drop('rank').index) #findme


In [23]:
len(features)

43

In [24]:
ss = StandardScaler()

knn = KNeighborsClassifier()

pipe = Pipeline([
    ('ss', ss),
    ('knn', knn)
])

In [25]:
params = {
    'knn__n_neighbors': [1, 2],
    'knn__p': [1],
    'knn__weights': ['uniform', 'distance']
}

gs = GridSearchCV(pipe, param_grid=params, cv=5)
gs.fit(X_train[features], y_train)
print(gs.best_params_)
print(gs.best_score_)

/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

{'knn__n_neighbors': 2, 'knn__p': 1, 'knn__weights': 'uniform'}
0.591768333556777


/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


In [26]:
predictions=gs.predict(X_train[features])
confusion_matrix(predictions, y_train)

array([[4037,    0,    4,    3,    2,    4,    0,    1,    2,    1,    2,
           2,    2,    3,    2,    3,    5,    5,    9,   11,   13,    4,
           5,    3,    1],
       [   0,  166,   32,   27,    9,   10,   12,    7,    5,    3,    2,
           0,    4,    3,    3,    1,    2,    0,    1,    0,    0,    0,
           0,    0,    0],
       [   0,    0,  130,   16,   17,   14,   12,    7,    4,    3,    9,
           2,    4,    4,    1,    2,    0,    0,    0,    1,    0,    0,
           0,    0,    0],
       [   0,    0,    0,  120,   19,   17,   10,    9,    7,    6,    7,
           5,    5,    0,    3,    5,    3,    0,    0,    0,    0,    1,
           0,    0,    0],
       [   0,    0,    0,    0,  119,   18,   22,   18,    6,   10,    7,
           3,    5,    4,    4,    2,    0,    0,    1,    0,    0,    0,
           0,    0,    0],
       [   0,    0,    0,    0,    0,  103,    9,   15,   10,   12,   10,
           7,    6,    4,    2,    2,    4,    0,  

In [29]:
print(y_holdout['rank'].value_counts())
y_holdout['rank'].value_counts(True)

0.0     1346
8.0       55
4.0       55
15.0      55
7.0       55
1.0       55
9.0       55
14.0      55
11.0      55
16.0      55
2.0       55
10.0      55
13.0      55
3.0       55
12.0      55
6.0       55
5.0       55
17.0      53
18.0      52
19.0      49
20.0      43
21.0      26
22.0      20
23.0      11
24.0       7
Name: rank, dtype: int64


0.0     0.541214
8.0     0.022115
4.0     0.022115
15.0    0.022115
7.0     0.022115
1.0     0.022115
9.0     0.022115
14.0    0.022115
11.0    0.022115
16.0    0.022115
2.0     0.022115
10.0    0.022115
13.0    0.022115
3.0     0.022115
12.0    0.022115
6.0     0.022115
5.0     0.022115
17.0    0.021311
18.0    0.020909
19.0    0.019702
20.0    0.017290
21.0    0.010454
22.0    0.008042
23.0    0.004423
24.0    0.002815
Name: rank, dtype: float64

In [30]:
gs.score(X_holdout[features], y_holdout)

0.584238037796542

In [31]:
predictions=gs.predict(X_holdout[features])
confusion_matrix(predictions, y_holdout)

array([[1330,    1,    4,    2,    1,    0,    4,    1,    2,    1,    2,
           3,    2,    2,    3,    4,    5,    3,    5,    4,    5,    5,
           0,    3,    0],
       [   1,   20,   14,   11,   11,    5,    5,    6,    3,    7,    5,
           1,    1,    0,    1,    0,    0,    1,    0,    0,    0,    0,
           0,    0,    0],
       [   0,   12,   11,    8,    4,    8,    2,    3,    6,    3,    2,
           6,    1,    0,    3,    0,    1,    0,    0,    0,    0,    0,
           0,    0,    0],
       [   0,    7,   11,   13,   18,   11,    4,    2,    1,    4,    2,
           3,    1,    4,    2,    0,    0,    0,    0,    2,    0,    0,
           0,    0,    0],
       [   0,    7,    2,    8,    5,    5,   12,    4,    0,    1,    2,
           4,    2,    0,    2,    1,    0,    1,    0,    0,    0,    0,
           0,    0,    0],
       [   0,    3,    2,    4,    2,    3,    6,    4,    4,    5,    3,
           4,    1,    2,    5,    2,    2,    1,  